In [2]:
import os 
import re 
import requests 
import pandas as pd 
import datetime 
import json
import time

In [3]:
# 请求头
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate ',
    'Accept-Language': 'zh-CN',
    'Connection':'keep-alive',
    'Content-Type':'application/json;charset=UTF-8',
    'Cookie': '__jsluid_h=e19768a3a74d5f0175cee66f84784f4a; wdcid=1187b9ab73353a36; 4de1d0bdb25d4625be2481a1b9e1350f=WyIyMDQ3MDc5Nzk3Il0; language=zh-CN; wdlast=1681700338; wdses=378cd3c4985bdd7e',
    'Host': 'liuyan.people.com.cn',
    'Origin': 'http://liuyan.people.com.cn',
    'Referer': 'http://liuyan.people.com.cn/threads/list?checkStatus=0&fid=733&formName=%E4%B8%9C%E5%9F%8E%E5%8C%BA%E5%A7%94%E4%B9%A6%E8%AE%B0%E5%AD%99%E6%96%B0%E5%86%9B&position=0&province=4&city=638&saveLocation=4&pForumNames=%E5%8C%97%E4%BA%AC%E5%B8%82&pForumNames=%E4%B8%9C%E5%9F%8E%E5%8C%BA',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48'}

In [6]:
import hashlib
def generation_signature(a,t,e="/v1/threads/list/df"):
    """
    a:appCode
    t:param
    """
    c=''
    s = hashlib.md5(a.encode()).hexdigest()[:16]
    text = e+t+s+c
    return hashlib.md5(text.encode()).hexdigest()

## 获取领导的id编号

In [7]:
t = '{"fid":72,"_t":'+ str(round(time.time()*1000)) + '}'
appCode = "PC42ce3bfa4980a9"
signature = generation_signature(appCode,t,e="/v1/forum/getChildForums")
params = {
  "appCode": appCode,
  "token": "",
  "signature": signature,
  "param": t
}
url = 'http://liuyan.people.com.cn/v1/forum/getChildForums?'
leader_result = requests.get(url, headers=headers, params=params).json()
leader_fids=[]
leader_name=[]
leader_tnumYear=[]
for i in leader_result['data']:
    leader_fids.append(i['fid'])
    leader_name.append(i['name'])
    leader_tnumYear.append(i['tnumYear'])

## 获取文本

In [223]:
def process_content(result):
    tid = []
    forumName = []
    subject = []
    typeName = []
    domainName = []
    stateInfo = []
    createDateline = []
    content = []
    for i in result:
        tid.append(i['tid'])
        forumName.append(i['forumName'])
        subject.append(i['subject'])
        typeName.append(i['typeName'])
        domainName.append(i['domainName'])
        stateInfo.append(i['stateInfo'])
        date_time = datetime.datetime.fromtimestamp(i['createDateline'])
        createDateline.append(date_time.strftime("%Y-%m-%d %H:%M:%S"))
        content.append(i['content'])
    df = pd.DataFrame({'序号':tid,'领导':forumName,'标题':subject,'类型':typeName,'领域':domainName,'状态':stateInfo,'创建日期':createDateline,'详细内容':content})
    return df
def continue_process(fid,appCode,lastItem,headers):
    param = '{\"fid\":\"' + str(fid) +'\",\"showUnAnswer\":1,\"typeId\":5,\"lastItem\":\"'+ str(lastItem) +'\",\"position\":\"0\",\"rows\":10,\"orderType\":2}'
    signature = generation_signature(appCode,param)
    params = {
        "appCode": appCode,
        "token": "",
        "signature": signature,
        "param": param}
    return requests.post(url, headers=headers,data=json.dumps(params)).json()

In [224]:
url = 'http://liuyan.people.com.cn/v1/threads/list/df'
for n,fid in enumerate(leader_fids):
    data_result = pd.DataFrame(data=None,columns=['序号','领导','标题','类型','领域','状态','创建日期','详细内容'])
    param = '{\"fid\":\"' + str(fid) +'\",\"showUnAnswer\":1,\"typeId\":5,\"lastItem\":\"\",\"position\":\"0\",\"rows\":10,\"orderType\":2}'
    signature = generation_signature(appCode,param)
    params = {
        "appCode": appCode,
        "token": "",
        "signature": signature,
        "param": param}
    result = requests.post(url, headers=headers,data=json.dumps(params)).json()
    df = process_content(result['resultData']['data'])
    data_result = pd.concat([data_result,df],axis=0,ignore_index=True)
    """
    通过最后一条留言的id爬取“加载更多”中的信息
    """
    while len(data_result) < leader_tnumYear[n]-1:
        result = continue_process(fid,appCode,result['resultData']['data'][-1]['tid'],headers)
        try:
            df = process_content(result['resultData']['data'])
        except:
            break
        data_result = pd.concat([data_result,df],axis=0,ignore_index=True)
        print('正在缓冲，等待下一次访问，目前已完成：'+ str(len(data_result)))
        time.sleep(10)
    data_result.to_csv(str(leader_name[n])+'.csv')
    print(str(leader_name[n])+'已完成')
    if n < len(leader_fids)-1:
        print('正在缓冲，等待下一次访问')
    time.sleep(10)
    break

正在缓冲，等待下一次访问，目前已完成：20
正在缓冲，等待下一次访问，目前已完成：30
正在缓冲，等待下一次访问，目前已完成：40
正在缓冲，等待下一次访问，目前已完成：50
正在缓冲，等待下一次访问，目前已完成：60
正在缓冲，等待下一次访问，目前已完成：70
正在缓冲，等待下一次访问，目前已完成：80
正在缓冲，等待下一次访问，目前已完成：90
正在缓冲，等待下一次访问，目前已完成：100
正在缓冲，等待下一次访问，目前已完成：110
正在缓冲，等待下一次访问，目前已完成：120
大同市委书记卢东亮已完成
正在缓冲，等待下一次访问
